In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.densenet import preprocess_input

import os
import cv2
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SEED = 1000
IMG_SIZE = 224
BATCH_SIZE = 32

TRAIN_DIR = '../input/caltech-zipped/Caltech Bird 2/Train'
VALID_DIR = '../input/caltech-zipped/Caltech Bird 2/Validation'
TEST_DIR = '../input/caltech-zipped/Caltech Bird 2/Test'

In [3]:
from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')

In [4]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 199


In [5]:
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications import DenseNet121

#IMG_SIZE=80

IMAGE_SIZE = [IMG_SIZE, IMG_SIZE] 


En = DenseNet121(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 


for layer in En.layers:
    layer.trainable = False

x = Flatten()(En.output)

x = Dense(num_classes, activation = 'softmax')(x)  

model = Model(inputs = En.input, outputs = x)

2022-02-13 08:20:05.655427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-13 08:20:05.776104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-13 08:20:05.776862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-13 08:20:05.778509: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

29097984/29084464 [==============================] - 0s 0us/step


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [7]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(
    training_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

Found 8190 images belonging to 199 classes.
Found 2311 images belonging to 199 classes.


In [8]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Learning rate:  0.001


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [10]:
h = model.fit(training_generator, 
              epochs=50,
              steps_per_epoch = 8190 // BATCH_SIZE,
              validation_data = validation_generator,
              validation_steps = 2311 // BATCH_SIZE,
              callbacks=lr_scheduler,
              verbose=2
             )

2022-02-13 08:20:12.272862: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
Learning rate:  0.001


2022-02-13 08:20:18.721132: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


255/255 - 196s - loss: 8.5833 - accuracy: 0.0132 - val_loss: 7.5900 - val_accuracy: 0.0326
Epoch 2/50
Learning rate:  0.001
255/255 - 130s - loss: 7.0979 - accuracy: 0.0520 - val_loss: 8.0650 - val_accuracy: 0.0430
Epoch 3/50
Learning rate:  0.001
255/255 - 134s - loss: 6.4109 - accuracy: 0.0979 - val_loss: 8.3997 - val_accuracy: 0.0373
Epoch 4/50
Learning rate:  0.001
255/255 - 128s - loss: 6.0960 - accuracy: 0.1252 - val_loss: 8.2450 - val_accuracy: 0.0512
Epoch 5/50
Learning rate:  0.001
255/255 - 132s - loss: 5.9182 - accuracy: 0.1573 - val_loss: 9.0894 - val_accuracy: 0.0404
Epoch 6/50
Learning rate:  0.001
255/255 - 131s - loss: 5.4168 - accuracy: 0.1936 - val_loss: 8.5204 - val_accuracy: 0.0356
Epoch 7/50
Learning rate:  0.001
255/255 - 132s - loss: 5.3054 - accuracy: 0.2150 - val_loss: 9.4963 - val_accuracy: 0.0438
Epoch 8/50
Learning rate:  0.001
255/255 - 132s - loss: 5.2252 - accuracy: 0.2265 - val_loss: 9.4044 - val_accuracy: 0.0473
Epoch 9/50
Learning rate:  0.001
255/255 